# Nagel–Schreckenberg model

Nagel–Schreckenberg model is a simple model used for modelling highway traffic.
#### Road
A road consists of cells. Each cell represents a single car or empty space on the road. 
#### Car 
Each car has a velocity (originally 0-5 units).
#### Transisition function
The transition function performs the following steps (order matters):
* Acceleration: The velocity of all cars having a velocity lower than the maximum velocity is increased by 1.
* Slowing down: If the distance between the current and the following car is smaller than the car's velocity, the velocity is reduced to the number of empty cells in front of the car (avoiding collisions).
* Randomization: The speed of all cars that have a *velocity >= 1* is reduced by one unit with the probability of *p*. 
* Car motion: All cars are moved forward by the *velocity* cells.

(source and more detailed description: [here](https://en.wikipedia.org/wiki/Nagel–Schreckenberg_model))

## Python implementation of Nagel-Schreckenberg model (6 points)

* Create a working Nagel-Schreckenberg model using the following template (4 points).
* Add periodic boundaries (1 point).
* Add a function that will randomly add new cars at the beginning of the road (1 point).

In [61]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import random
from IPython.display import HTML

In [66]:
%%capture
# Parameters
roadLength = 50
minVelocity = 0
maxVelocity = 5

# Number of steps 
steps = 300

rand_probability = 0.5 

# Config
fig, ax = plt.subplots(figsize=(12,4))
plt.axis('off')

In [63]:
class Car:
    distance = 0

    
    def __init__(self, minVelocity, maxVelocity):
        import random
        self.color = random.randint(0,200)
        self.velocity = random.randint(minVelocity,maxVelocity)
    
    #TODO Implement increaseVelcoity and randomizeVelocity functions 
    def increaseVelocity(self,maxVelocity):
        self.velocity = min(self.velocity + 1, maxVelocity)
    
    def setVelocity(self, newVelocity,maxVelocity):
        if newVelocity <= maxVelocity:
            self.velocity = newVelocity
        
    def randomizeVelocity(self, probability):
        diff = random.choices((1, 0), weights=(probability, 1.0 - probability))[0]
        self.velocity = max(minVelocity, self.velocity - diff)
        
    def getColor(self):
        return self.color
    
    def isEmpty(self):
        return false
    
    def setDistance(self,distance):
        self.distance = distance

class Road:
    def __init__(self):
        return
    def getColor(self):
        return 255
    def isEmpty(self):
        return true

In [64]:
def initGrid(roadLength):
    # TODO Randomly initialize grid here (remove existing code)
    # grid should be a list
    seed = random.choices((1, 0), weights=(1, 10), k=roadLength)
    grid = [Road() if s == 0 else Car(minVelocity, maxVelocity) for s in seed]

    return grid

def show(grid, im):
        im.set_data(np.array(list(map(lambda x : x.getColor(), grid))).reshape(1,roadLength))


def update(frameNum, grid, im, roadLength):
    if(frameNum == 0):
        show(grid,im)
        
    else:
        # 1.Acceleration
        for i in grid:
            if type(i) is Car:
                i.increaseVelocity(maxVelocity)

        # 2. Slow down

        # TODO Implement slow down
        for idx, cell in enumerate(grid):
            if type(cell) is Car:
                space_ahead = 0
                
                j, N = idx + 1, len(grid)
                while type(grid[j % N]) is not Car:
                    j += 1
                space_ahead = j - idx - 1
                cell.velocity = max(min(cell.velocity, space_ahead), minVelocity)


        # 3. Randomization
        for i in grid:
            if type(i) is Car:
                i.randomizeVelocity(rand_probability)


        # 4. Car motion
        newGrid = [Road()] * roadLength

        # TODO Implement motion
        for idx, cell in enumerate(grid):
            if type(cell) is Car:
                new_idx = idx + cell.velocity
                newGrid[new_idx % len(newGrid)] = cell

        # Add new vehicles.
        if all(type(c) is Road for c in newGrid[-2:]) and all(type(c) is Road for c in newGrid[:2]) and random.choices((1, 0), weights=(1, 4)):
            newGrid[0] = Car(minVelocity, maxVelocity)
                
        grid[:] = newGrid[:]
        
        show(grid, im)
    return im,



In [67]:
grid = initGrid(roadLength)
im = ax.imshow(np.array(list(map(lambda x : x.getColor(), grid))).reshape(1,roadLength),cmap = 'gist_ncar')
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [grid, im, roadLength])

HTML(ani.to_jshtml())


